In [1]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
import struct
import hashlib
import pandas as pd
import numpy as np

In [2]:
class Block:
    def __init__(self, data, prev_hash, encryption_key=None):
        self.data = data
        self.prev_hash = prev_hash
        self.hash = self.calc_hash()
        self.encryption_key = encryption_key

    def calc_hash(self):
        sha = hashlib.sha256()
        sha.update(self.data.encode('utf-8'))
        return sha.hexdigest()

    def encrypt_data(self):
        if self.encryption_key:
            numeric_array = self.data
            message_bytes = b''.join(struct.pack('!Q', num) for num in numeric_array)
            encrypted_message = self.encryption_key.encrypt(
                message_bytes,
                padding.OAEP(
                    mgf=padding.MGF1(algorithm=hashes.SHA256()),
                    algorithm=hashes.SHA256(),
                    label=None
                )
            )
            self.data = encrypted_message.hex()

    def decrypt_data(self):
        if self.encryption_key:
            encrypted_message = bytes.fromhex(self.data)
            decrypted_message_bytes = self.encryption_key.decrypt(
                encrypted_message,
                padding.OAEP(
                    mgf=padding.MGF1(algorithm=hashes.SHA256()),
                    algorithm=hashes.SHA256(),
                    label=None
                )
            )
            self.data = [struct.unpack('!Q', decrypted_message_bytes[i:i+8])[0] for i in range(0, len(decrypted_message_bytes), 8)]

class Blockchain:
    def __init__(self):
        self.chain = [self.calc_genesis_block()]

    def calc_genesis_block(self):
        return Block("Genesis Block", "0")

    def add_block(self, data, encryption_key=None):
        prev_block = self.chain[-1]
        new_block = Block(data, prev_block.hash, encryption_key)
        new_block.encrypt_data()  # Encrypt data before adding it to the block
        self.chain.append(new_block)

    def decrypt_block_data(self, block_index, decryption_key):
        if block_index >= 0 and block_index < len(self.chain):
            block = self.chain[block_index]
            block.encryption_key = decryption_key
            block.decrypt_data()

In [4]:
zone_1_data = pd.read_csv(
    r"C:\Users\arshi\OneDrive\Desktop\PH.D\Blockchain\data\Sz1n.csv")

data_to_encrypt = np.array(zone_1_data)

# Create a blockchain instance
my_blockchain = Blockchain()

# Generate an RSA key pair
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)

public_key = private_key.public_key()

# Serialize the public key for future use (e.g., sharing with others)
public_key_bytes = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

# Encrypt and decrypt each block's data and print the results
for block_index in range(len(data_to_encrypt)):
    block_data = data_to_encrypt[block_index]

    # Encrypt the block's data
    encrypted_message = public_key.encrypt(
        struct.pack('f' * len(block_data), *block_data),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    # Decrypt the block's data
    decrypted_bytes = private_key.decrypt(
        encrypted_message,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    # Unpack the decrypted bytes back into a numeric array
    decrypted_array = struct.unpack('f' * (len(block_data)), decrypted_bytes)

    # Print the results
    print(f"Block {block_index + 1}:")
    print("Original Numeric Array:", block_data)
    print("Encrypted Message:", encrypted_message.hex())
    print("Decrypted Numeric Array:", decrypted_array)
    print()

Block 1:
Original Numeric Array: [  2.427057 522.8467     2.         2.         1.      ]
Encrypted Message: 4812191c58f33b525a0d672f95b803ab6731df5a59cbe461801a9ee34754a86b82d722cab6c08e57b0bdef6cc091a20b55f1c5603a12380a031ba12da8f1add6f167040c310434d90935b5b7421149a687b831bad757c9aedaed427facd3048c8da5a64aa5a5c0b569a2fdac709f79fc5139a3278f2130e30a3c0c4c2d41e6f8c37fd926e524be9cf4a82606953e953e6b3cd61f787583f23fc7ffef5e4f0c8f19efa9f64210467b5b4fcf490f58051f5c307e96e2832c74b8292bd7d9ce3deb32577e31344f34a3e0f3f7d397f24ea823f21da70ba867c22364a38df661acabd24e04a3a76f3ca023ff92d7b3f2948e0bf41e0df13e392a56f8953866c50836
Decrypted Numeric Array: (2.4270570278167725, 522.8466796875, 2.0, 2.0, 1.0)

Block 2:
Original Numeric Array: [  2.446274 522.886      2.         2.         1.      ]
Encrypted Message: 3596f8ee69e07ae8cfccb0efe492e714375dc6c39d6d270ce1fb410e79985573403824db43b84032d2ec9a2ac904b7991c7cf91c4c95e62afa424b14c489cddfd277239807853f7354901681c43b5526d100803228529a514c3522cc5503ccf